In [1]:
import os

# Set the folder path in Kaggle input directory
drive_folder_path = '/kaggle/input/wav-file/'  # Update with your actual folder name

# List available files in the folder
audio_files = [f for f in os.listdir(drive_folder_path) if f.endswith('.wav')]

if not audio_files:
    raise ValueError("No .wav files found in the folder.")

# Display files to the user for selection
print("Available audio files in the folder:")
for idx, file_name in enumerate(audio_files, start=1):
    print(f"{idx}. {file_name}")

# Let the user select the file (this part needs to be adapted for batch processing or direct selection)
selected_index = int(input("Enter the number corresponding to the file you want to process: ")) - 1

if selected_index < 0 or selected_index >= len(audio_files):
    raise ValueError("Invalid selection.")

# Set the selected file path
path = os.path.join(drive_folder_path, audio_files[selected_index])
print(f"Selected file: {path}")

# Output folder for results (set to a folder within the Kaggle environment)
output_folder_path = '/kaggle/working/'  # You can set a custom folder if needed


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/wav-file/'

In [235]:
# Directory to save segmented audio files in Google Drive

segmented_audio_folder = os.path.join(output_folder_path, "segmented_audio")

os.makedirs(segmented_audio_folder, exist_ok=True)  # Create the folder if it doesn't exist

In [3]:
# # Parameters

# num_speakers = 2  #@param {type:"integer"}

# language = 'any'  #@param ['any', 'bengali']

# model_size = 'large'  #@param ['tiny', 'base', 'small', 'medium', 'large', 'bengali']

In [7]:
threshold = 10

In [5]:
!pip install transformers==4.32.0

In [ ]:
# model_name = model_size

# if language == 'bengali' and model_size != 'medium':

#     model_name += '.bn'

In [4]:
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

The system cannot find the path specified.


In [12]:
from pydub import AudioSegment

from io import BytesIO

import glob

import uuid

import asyncio

from transformers import pipeline



import torch,os

torch.cuda.empty_cache()

CHUNK_LENGTH_S = 8

DATASET_PATH = os.getcwd()+"/audios"

unique_name = str(uuid.uuid4())

os.makedirs(DATASET_PATH, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = None

model_name = "maddi99/whisper_bn"

pipe = pipeline(task="automatic-speech-recognition",

                model=model_name,

                tokenizer=model_name,

                chunk_length_s=CHUNK_LENGTH_S, device=0)

pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="bn",

                                                                                 task="transcribe")

In [16]:
# # import libraries

# import os

# # import whisper

# import torch

from pydub import AudioSegment

from pyannote.audio.pipelines import VoiceActivityDetection as VoiceActivitySegmentation

from pyannote.audio import Model

from pyannote.core import Segment, Annotation, Timeline

from sklearn.cluster import AgglomerativeClustering

import numpy as np

from tqdm import tqdm

import urllib.request

from pyannote.audio.core.io import AudioFile

from pydub.silence import detect_nonsilent

from pyannote.audio import Model

from pyannote.audio import Audio

from pyannote.audio.pipelines import VoiceActivityDetection as VoiceActivitySegmentation

In [220]:
# Define VAD segmentation model URL

VAD_SEGMENTATION_URL = "https://whisperx.s3.eu-west-2.amazonaws.com/model_weights/segmentation/0b5b3216d60a2d32fc086b47ea8c67589aaeb26b7e07fcbe620d6d0b83e209ea/pytorch_model.bin"



# Function to load VAD model with optimized hyperparameters

def load_vad_model(device, vad_onset=0.5, vad_offset=0.3, model_fp=None):

    model_dir = torch.hub._get_torch_home()

    os.makedirs(model_dir, exist_ok=True)

    if model_fp is None:

        model_fp = os.path.join(model_dir, "whisperx-vad-segmentation.bin")

    if not os.path.isfile(model_fp):

        with urllib.request.urlopen(VAD_SEGMENTATION_URL) as source, open(model_fp, "wb") as output:

            with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit="iB", unit_scale=True, unit_divisor=1024) as loop:

                while True:

                    buffer = source.read(8192)

                    if not buffer:

                        break

                    output.write(buffer)

                    loop.update(len(buffer))



    vad_model = Model.from_pretrained(model_fp)

    vad_pipeline = VoiceActivitySegmentation(segmentation=vad_model, device=torch.device(device))

    vad_pipeline.instantiate({

        "onset": vad_onset,

        "offset": vad_offset,

        "min_duration_on": 0.5,  # Discard segments shorter than 1 second

        "min_duration_off": 0.3  # Merge gaps shorter than 0.5 seconds

    })

    return vad_pipeline


In [236]:
# Load VAD model

device = "cuda" if torch.cuda.is_available() else "cpu"

vad_pipeline = load_vad_model(device)

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.0. Bad things might happen unless you revert torch to 1.x.


In [237]:
# Convert non-WAV files to WAV

if path[-3:] != 'wav':

    subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])

    path = 'audio.wav'


In [238]:
print(path)

/kaggle/input/wav-file/20241102_105316_CMEDHLTD_dr_aamasud_01732928315.wav


In [250]:
# Apply VAD to detect speech segments

vad_segments = vad_pipeline.apply({"audio": path})

In [251]:
print(vad_segments)

[ 00:00:12.872 -->  00:00:16.939] A SPEECH
[ 00:00:17.395 -->  00:00:17.969] B SPEECH
[ 00:00:20.297 -->  00:00:22.727] D SPEECH
[ 00:00:23.605 -->  00:00:28.448] E SPEECH
[ 00:00:28.752 -->  00:00:35.569] F SPEECH
[ 00:00:36.261 -->  00:00:38.978] G SPEECH
[ 00:00:39.653 -->  00:00:51.212] H SPEECH
[ 00:00:51.634 -->  00:00:54.807] I SPEECH
[ 00:00:55.380 -->  00:00:56.393] J SPEECH
[ 00:00:57.085 -->  00:01:05.084] K SPEECH
[ 00:01:05.894 -->  00:01:08.459] L SPEECH
[ 00:01:08.864 -->  00:01:21.115] M SPEECH
[ 00:01:22.583 -->  00:01:26.616] N SPEECH
[ 00:01:27.696 -->  00:01:32.792] O SPEECH
[ 00:01:34.075 -->  00:01:36.319] P SPEECH
[ 00:01:36.944 -->  00:01:42.749] Q SPEECH
[ 00:01:43.457 -->  00:01:58.780] R SPEECH
[ 00:02:00.450 -->  00:02:05.462] S SPEECH
[ 00:02:06.104 -->  00:02:09.732] T SPEECH


In [ ]:
# print(merged_segments)

In [252]:
timeline = vad_segments.get_timeline()

print(timeline)

[[ 00:00:12.872 -->  00:00:16.939]
 [ 00:00:17.395 -->  00:00:17.969]
 [ 00:00:20.297 -->  00:00:22.727]
 [ 00:00:23.605 -->  00:00:28.448]
 [ 00:00:28.752 -->  00:00:35.569]
 [ 00:00:36.261 -->  00:00:38.978]
 [ 00:00:39.653 -->  00:00:51.212]
 [ 00:00:51.634 -->  00:00:54.807]
 [ 00:00:55.380 -->  00:00:56.393]
 [ 00:00:57.085 -->  00:01:05.084]
 [ 00:01:05.894 -->  00:01:08.459]
 [ 00:01:08.864 -->  00:01:21.115]
 [ 00:01:22.583 -->  00:01:26.616]
 [ 00:01:27.696 -->  00:01:32.792]
 [ 00:01:34.075 -->  00:01:36.319]
 [ 00:01:36.944 -->  00:01:42.749]
 [ 00:01:43.457 -->  00:01:58.780]
 [ 00:02:00.450 -->  00:02:05.462]
 [ 00:02:06.104 -->  00:02:09.732]]


In [253]:
# Function to transcribe with optional chunking

def transcribe_with_chunking(segment_path, max_chunk_duration=threshold):

    audio = AudioSegment.from_file(segment_path)

    duration = len(audio) / 1000  # Duration in seconds



    if duration <= max_chunk_duration:

        # Short segment, transcribe directly

        result = pipe(segment_path)

        return result["text"]

    else:

        # Long segment, manually chunk the audio

        chunks = []

        for start in range(0, int(duration), max_chunk_duration):

            end = min(start + max_chunk_duration, duration)

            chunk_audio = audio[start * 1000:end * 1000]

            chunk_path = f"temp_chunk_{start}_{end}.wav"

            chunk_audio.export(chunk_path, format="wav")



            # Transcribe each chunk

            chunk_result = pipe(chunk_path)

            chunks.append(chunk_result["text"])



        # Combine transcriptions from all chunks

        return " ".join(chunks)

In [243]:
# Load Whisper model

# whisper_model = whisper.load_model(model_name)

model = model_name

In [254]:


# Transcribe segments with chunking

def transcribe_with_chunking(segment_path, max_chunk_duration=threshold):

    audio = AudioSegment.from_file(segment_path)

    duration = len(audio) / 1000  # Duration in seconds



    if duration <= max_chunk_duration:

        result = pipe(segment_path)

        return result["text"]

    else:

        chunks = []

        for start in range(0, int(duration), max_chunk_duration):

            end = min(start + max_chunk_duration, duration)

            chunk_audio = audio[start * 1000:end * 1000]

            chunk_path = f"temp_chunk_{start}_{end}.wav"

            chunk_audio.export(chunk_path, format="wav")

            chunk_result = pipe(chunk_path)

            chunks.append(chunk_result["text"])

        return " ".join(chunks)

In [ ]:
# # Function to merge short segments

# def merge_segments(vad_segments, min_length=3.0, max_gap=1.0):

#     timeline = vad_segments.get_timeline().support(collar=max_gap)

#     merged_timeline = Timeline()

#     for segment in timeline:

#         if segment.duration >= min_length:

#             merged_timeline.add(segment)

#     return merged_timeline



# # Merge VAD segments

# merged_segments = merge_segments(vad_segments, min_length=3.0)

In [255]:
# Process and save each segment

for i, segment in enumerate(timeline):

    start_ms = segment.start * 1000  # Convert segment start to milliseconds

    end_ms = segment.end * 1000  # Convert segment end to milliseconds

    segment_audio = AudioSegment.from_file(path)[start_ms:end_ms]  # Extract audio segment

    segment_filename = f"segment_{i+1}.wav"  # Name for the segment file

    segment_path = os.path.join(segmented_audio_folder, segment_filename)

    segment_audio.export(segment_path, format="wav")  # Save segment as a .wav file



    # Transcribe the segment

    transcription = transcribe_with_chunking(segment_path, max_chunk_duration=threshold)

    print(f"Transcription for segment {i+1}: {transcription}")

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcription for segment 1: হ্যালো আন্তোনা প্রদীপ বলছেন


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcription for segment 2: জি স্যার


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

Transcription for segment 17:  একজনের চিকিৎসা না করাইলে ভালো হবে না এটা আরেকজনের কাছ থেকে আবার আরেবর যেগুলো আছে বালিশের চাদর বিছানার কভারের বিষয়ে গুলা যাবত পরাজয় করতেন সেই কাপড়গুলা গরম পানিতে টুল পাল্টাতে ভালো করে ধুয়ে নিবে


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcription for segment 18:  আর হচ্ছে ওষুধ দিচ্ছি ওষুধের গুলা আপনার আপনি আপনার মা দুজনেই স্টেম্বারে ব্যবহার করবেন ঠিক আছে


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcription for segment 19:  ওকে ঠিক আছে সুশীল করে পাঠাইছে


In [258]:
num_speakers = 2

In [259]:
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device=torch.device(device))

audio = Audio()



def segment_embedding(segment):

    start = segment.start

    end = segment.end

    waveform, sample_rate = audio.crop(path, Segment(start, end))

    return embedding_model(waveform[None])



embeddings = np.zeros((len(timeline), 192))

for i, segment in enumerate(timeline):

    embeddings[i] = segment_embedding(segment)



clustering = AgglomerativeClustering(n_clusters=num_speakers).fit(embeddings)

labels = clustering.labels_


/opt/conda/lib/python3.10/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [260]:
# Write diarized transcription to a file in Google Drive

# output_file_path = os.path.join(output_folder_path, "transcript2.txt")

# with open(output_file_path, "w",  encoding="utf-8") as f:

#     for i, segment in enumerate(timeline):

#         f.write(f"\nSPEAKER {labels[i] + 1} {segment.start}-{segment.end} sec:\n")

#         f.write(transcribe_with_chunking(os.path.join(segmented_audio_folder, f"segment_{i+1}.wav"), max_chunk_duration=threshold) + "\n")

audio_file_name = os.path.splitext(os.path.basename(path))[0]

# Use the audio file's name for the transcript file
output_file_path = os.path.join(output_folder_path, f"{audio_file_name}.txt")

with open(output_file_path, "w", encoding="utf-8") as f:
    for i, segment in enumerate(timeline):
        f.write(f"\nSPEAKER {labels[i] + 1} {segment.start}-{segment.end} sec:\n")
        f.write(transcribe_with_chunking(
            os.path.join(segmented_audio_folder, f"segment_{i+1}.wav"), 
            max_chunk_duration=threshold) + "\n")

print(f"Transcription with speaker labels saved at: {output_file_path}")
print(f"Segmented audio files saved in folder: {segmented_audio_folder}")








/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentiall

Transcription with speaker labels saved at: /kaggle/working/20241102_110623_CMEDHLTD_dr_aamasud_01610895014137.txt
Segmented audio files saved in folder: /kaggle/working/segmented_audio
